<a href="https://colab.research.google.com/github/thakare-om03/AyurMed/blob/main/RUNTIME_TERROR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT NECESSARY LIBRARIES**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import sys
sys.path.append('/path/to/data_preprocessing')

# **DEFINE** **FUNCTIONS**

In [ ]:
def load_and_preprocess_data("/content/sample_data/AyurMed.csv"):
    data = pd.read_csv("/content/sample_data/AyurMed.csv")
    inputs = data.drop(['symptoms','remedy type'],axis=1)
    target = inputs['Cure']
    inputs = inputs.drop(columns=['Disease', 'gender', 'diabetic', 'age', 'prakruti','remedies','Cure'])

    label_encoders = {}
    for column in ['Disease', 'gender', 'diabetic', 'prakruti', 'remedies', 'age']:
        le = LabelEncoder()
        le.fit(data[column])
        inputs[column] = le.transform(data[column])
        label_encoders[column] = le

    return inputs, target, label_encoders

def train_model(inputs, target):
    model = XGBClassifier(
        objective="multi:softmax",
        num_classes=len(target.unique()),
        max_depth=None,
        learning_rate=0.1,
        n_estimators=50
    )
    model.fit(inputs, target)
    return model

def predict(model, new_data, label_encoders):
    for column in ['Disease', 'gender', 'diabetic', 'prakruti', 'age']:
        new_data[column] = label_encoders[column].transform(new_data[column])
    prediction = model.predict(new_data)
    remedy = label_encoders['remedies'].inverse_transform(prediction)
    return remedy

def plot_feature_importance(model):
    plt.figure(figsize=(12, 6))
    plt.title("Feature importance")
    plt.barh(inputs.columns, model.feature_importances_)
    plt.show()

SyntaxError: invalid syntax (<ipython-input-15-2747423f2a0e>, line 1)

# **LOAD AND PREPROCESS DATA**

In [ ]:
inputs, target, label_encoders = data_preprocessing.load_and_preprocess_data("/content/sample_data/AyurMed.csv")

NameError: name 'data_preprocessing' is not defined

# **TRAIN THE MODEL**

In [ ]:
AyurMed = train_model(inputs, target)

# **MAKE PREDICTION**

In [ ]:
remedy = predict(AyurMed, new_data, label_encoders)
print('The treatment is :', remedy[0])

# **PLOT FEATURE**

In [ ]:
plot_feature_importance(AyurMed)